***
## FIT5196 Assessment 1: Text Processing
Student Name: Devaneek sharma
Student ID:29781965
Date:14/04/2020

Environment: Python 3.6.5 and Jupyter notebook Libraries used:
            1. re (for regular expression, included in Anaconda Python 3.6)
            2. PyPDF2 (PdfFileReader)
            3. nlkt (probability,MWETokenizer tokenization and counter,porter stemmer)
            4. itertool(for tool)
            5. Collection(for counter).
            6. sklearn.feature_extraction.text(CountVectorizer)
            7. ast( for evaluating list as string)
     
***
## Introduction:
***
- **Step-1:** Preprocessing the PDF file is done i.e Text is been extracted from the pdf file using "PyPDF2" and pdf reader then by using the "re" library is used to extract outcome and snopysis.
- **Step-2:** Tokens are normalized to lowercase except the capital tokens appeared in the middle of a sentence/line.After preprocessing and cleaning of the data.   
- **Step-3:** For word tokenization the following regular expression,"\w+(?:[-']\w+)?" and RegexpTokenizer library is used. 
- **Step-4:** In this step first 200 meaningful bigrams are take out by using nltk colocations and are included in the vocab.Then, by using "mwetokenizer" again tokenisation is been done and adding to the collocation dictionary.  
- **Step-5:** The context-independent and context-dependent with the threshold set to %95 I.e words frequency>190 and word frequesncy<10 stop words are removed from the vocab dictionary. The stop words list from stopword provided by text is removed.In this step Unit wise frequency of the words is been counted I.e counting the individual words coming in how many unit. And for the words coming in more than 190 units and less than 10(rare word) from units are removed.
- **Step-6:** Tokens are stemmed using the Porter stemmer. which remove commoner linguistics and inflexional endings
- **Step-7:** Tokens with the length less than 3 should are removed from the vocab. as the porter stemmer decrease the the length of the words thus this step is done after stemming.
- **Step-8:** Vocab is then sorted in the alphabatical order and idexing to vocab is given. our vocab is now ready for unit parse representation.
- **Step-9:** Unit parse representation with the format is been created.
           
***


## Libraries used in the module.
- PyPDF2- used to convert the pdf to text
- Collections- used for counter, which is used to count number of units.
- nltk - tokenize for regex tokenizer, nltk.probability for frequency distribution, MWE tokenizer,porterstemmer.
- sklearn.feature_extraction.text -for count vectorizer.
- itertools- for chain which is used to get the list from the dictonary by chain looping.
- Ast evaluate the the list as the string 
***

In [ ]:
from PyPDF2 import PdfFileReader
import collections
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from itertools import chain
from nltk.probability import *
from nltk.tokenize import MWETokenizer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import ast

***
## Text extraction from PDF and preprocessing the text.

- PyPDF2, tool is used to extract the text from the file. After extraction text is been processed by using re library and regex is been written to filter unit,outcome and snopysis from the individual unit from the text.


Finally dictionary-"textDict" is been created where the key is the unitcode and have the textual value of outsome and snopysis.
thus this dictionary contains all the units with their value(outcome/synopsis).


***

In [ ]:
textDictionary={}
snopList=[]
outcomeList=[]
# plese set the path in order to run the script.
path = '29781965.pdf'
# regex to take the output and synopsis, this this full block of text for each of the uit
regexExtUnit= r"([A-Z]{3}[0-9]{4})((?:\n.*?)*])"
unitCodeRegex= r"([A-Z]{3}[0-9]{4})\n[A-Z]"
# to take outocme
regOutcome=r"(\['.*[\s\S]*\])"
# to take synopsis out.
regSnopsis=r"((?:.*\n)*)\['"
with open(path, 'rb') as f:
    # read the pdf file
    pdf = PdfFileReader(f)
    count = PdfFileReader.numPages
    out_dict = dict()
    # read the no of pages 
    for i in range(63):
        page = pdf.getPage(i)
        text = page.extractText()
        # using the regex to clear the data
        reg = re.findall(regexExtUnit,text)
        #regex to extract all the snopysis the and otucome
        for reg in re.findall(regexExtUnit,text):
            regValue= reg[1]
            key= reg[0]
            # regex to extract the snopysis and outcome.
            snop=re.findall(regSnopsis,regValue)                
            outcome= re.findall(regOutcome,regValue)
            temp = (outcome[0].replace("\n"," "))
            outcome_lines = ""
            # here the list is been treating as string.
            for x in ast.literal_eval(temp):
                outcome_lines += (x)
                # checking if the line end with'.' or not and the add '.' if not
                if not x.endswith('.'):
                    outcome_lines += "."
            spout = snop[0].replace("\n"," ")
            spout = spout.strip()
            spout += outcome_lines
            textDictionary[key] = spout
    
    print(textDictionary)   
            
      
            
            


***
## Normalisation:
- After the text is extracted and processed, function- "lowerCaseSentence" is been called that convert the forst words of the sentence and line to the lower case. from the "synopsis" delimiter used is '.' and for the outcome "','" is used as the delimiter.
- Normalisation is difficult to done once we do the tokenisation, as it would be difficult to find which token is in start of the line and sentence hence Normalisation is soon done after prepocessing the text
***


In [ ]:
#iterating through the items of dictionary
for unit, value in textDictionary.items():
    linesUnit = ""
    # spliting the line with delimiter '.'
    for line in value.split('.'):
        #cheking if the line is not null
        if line is not None:
            #split the line into words
            wordList = line.split()
            if len(wordList) != 0:
                #changes the words to lower case
                lowerWord = wordList[0].lower()
                otherLine = " ".join(wordList[1:])
                linesUnit += " " + lowerWord + " " + otherLine
    textDictionary[unit] = linesUnit
print(textDictionary)


***
## Tokenisation
- This is the helper function which is used to tokenise by using the regex tokenisation. expression r"\w+(?:[-']\w+)?"  has used to tokenise.
- once the token is been tokenise the tokens we get are the unigrams. 
- tokeniseData- this dictionary is been created which will have all the tokens for the individual unit.


#### Function: TokeniseText()
function return "unit id", and "tokens" for that unit.
#### parameters:
- unitId(string)- unit id for retrive the value from dictionary.

## Uni-gram:
- Token generated after the tokenisation are called as unigrams.
- chain.iterable is been used later to get the value of dictionary "tokeniseData" and then the total unigrams is been calculated.
- removing the alphanumeric words if any.
***

In [ ]:
# Tokenising the outcome and snopsis text 
def TokeniseText(unitId):
    #print(unitId)
    value = textDictionary[unitId]
    # regex tokeniser used to get the tokens
    tokenisedData = RegexpTokenizer(r"\w+(?:[-']\w+)?")
    tokens=tokenisedData.tokenize(value)
    return (unitId, tokens)

# tokenise data for each unit
tokeniseData = dict(TokeniseText(unitId) for unitId in textDictionary.keys())
unitTokens = list(chain.from_iterable(tokeniseData.values()))
print("Totale no of unigrams: ",len(unitTokens))


***
## Bigram generation
- Bigram has to be generated. we could find all the BI-gram by using bi-gram finder using the nltk. 
- Bigram is been generated before  context dependent and context independant stop words as some of uni-gram combining together could form the meaning word which could be lost if we are removing the context dependent and context independant first.
- words length than <3 will cause a same thing we may lose some meaningful bigrams
- same in the case of the removal of stemmer(as stemmer could inflexional the word). 
***

In [ ]:
#collocations finding the Bigram in the token.
# generating the Bigram for the corelation
bigramUnits = nltk.collocations.BigramAssocMeasures()
# passing the token to bigrms tokeniser
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unitTokens)
# by bigrams finder we could remove replace the bigrams.
top200Bigrams = bigramFinder.nbest(bigramUnits.pmi, 200)

#tokenising after bigram
mwetokenizer = MWETokenizer(top200Bigrams)
# adding the bigrms to the ourvocab dictionary.
collocationUnitsDict =  dict((unitId, mwetokenizer.tokenize(info)) for unitId,info in tokeniseData.items())
# getting list of our vocan dictionary.
allWordsCollocation = list(chain.from_iterable(collocationUnitsDict.values()))
print("After bigrams, length of our vocab: ", len(allWordsCollocation))
print(top200Bigrams)

***
## Removing Stop words
- Now that we have unigrms and bigrams, we could start removing the stop words presented in the file 'stopwords_en.txt'.
- Stop words is been read and then from the dictionary "collocationUnitsDict" which is out main dictionary, stopwords is been removed.
- As, stemmer could change the wrord which could potential be a stopword thus stop words are removed before doing the stemmer.
- After bigrams could form a ***Context Independent *** stop word thus after bigram, stop words is been removed.
***

In [ ]:
#removing the stop words
stopwords = []
with open('stopwords_en.txt') as f:
    stopwords = f.read().splitlines()
    #tokenizedUnits = {}

# removing the context independent stopwords from the out vocb dictionary.
for unitid in textDict.keys():
    # removing stop words from the dictionary
    collocationUnitsDict[unitid] = [wrd for wrd in collocationUnitsDict[unitid] if wrd not in stopwords]
# list of values of the vocab dictionary.
aftRemovingStopWords = list(chain.from_iterable(collocationUnitsDict.values()))
print("After removing context Independent stop words our vocab dictionay length: ",len(aftRemovingStopWords))
fd_3 = FreqDist(aftRemovingStopWords)
fd_3.most_common(50)



***
## Removing context dependant stopwords 
- After the context Independant, Context dependant stops words with threshold of greter than 190(95%) and less than 10(5%) is been removed.
- Here unit wise the frequency of the words is been calculated. I.e how many times individual word is coming in diffrent units.
- For the words more than 190 and less than 10 is been removed.
- As the stemmer change the words thus we need to remove the context dependant stopords before it. 
- In our data analysis if the word is coming in more than 190 it means that it is context dependant and e cant derive anything from it. same incase for words which is coming in less than 10 words, as we can't derive anything from them too.
***

In [ ]:
#To find the word frequency for idividual units-context dependant
dictUnitCount={}
# iterating through the key of the dictionary
for k,v in collocationUnitsDict.items():
    # used to count the no of individual words for each unit. 
    counter = Counter(collocationUnitsDict[k])
    # ading the count of word frequency for each unit to the new dictionary
    dictUnitCount[k]=counter
# getting the values for the list dicUnitcount
dic = list(chain.from_iterable(dictUnitCount.values()))
# calculate the frequency distribution the words. which will give how many times a single words come in different units.
freqDistriContDep= FreqDist(dic)
#freqDistriContDep.most_common(50)
#print(len(aftRemovingStopWords))

#removing the words with the count <10 or >190 and 
for unitid in collocationUnitsDict.keys():
    # iterating through the new dictionary containing freq distibution. 
    for k,v in freqDistriContDep.items():
        if v<10 or v>190:
            # removing the context dependant stopwords.
            collocationUnitsDict[unitid] = [wrd for wrd in collocationUnitsDict[unitid] if wrd!=k]

print("Length of vocab after removing Context dependant stopwords:",(len(list(chain.from_iterable(collocationUnitsDict.values())))))                     

***
# # Removing the words with the length less than 3.
Words with the length than 3 is been removed. this is been done before stemming as non-meaningful words after stemming could form different meaning and will not help in data analysis. thus words lenth less than 3 should be removed before stemming the words.
***

In [ ]:
#Removing the words having length less than 3:
for unitid in collocationUnitsDict.keys():
    collocationUnitsDict[unitid] = [wrd for wrd in collocationUnitsDict[unitid] if len(wrd)>=3]
               
#print("Length of vocab after removing Context dependant stopwords:",(len(list(chain.from_iterable(collocationUnitsDict.values())))))                     

***
# Extracting the Nouns.
Nouns words in the dictionary is been extracted from out vacab and not to be used in stemming. As for stemmer it changes the words meaning there is not point of stemming the noum.
***

In [ ]:
# list of vacab dictionary is been created for extracting the Nouns
listValueVocabDict = list(chain.from_iterable(collocationUnitsDict.values()))
# passing the vocab list for noun Part-of-Speeching tagging.
nounVocab = nltk.tag.pos_tag(listValueVocabDict)
#all the nouns is been created.
allNouns = [w for w,t in nounVocab if t.startswith('NN')]
allNouns


***
# Porter Stemmer
- Now that we have removed stop words, 
- Many variation of the word could contain the same meaning. Thus it is important to use stemmer and remove those words. As they would have no impact in the data analysis. 
- Nouns are not stemmed here.
***

In [ ]:
# use of the Porter stemmer
# porter stemmer object
ps = PorterStemmer()
#iterate through the vocab dictionary items
for key,value in collocationUnitsDict.items():
    stemOutputLst=[]
    # iterate throught the each unit list.
    for val in value:
        # checking if the words are nouns, if not stemmed them
        if val not in allNouns:
            # stemming the text
            stemOutput= ps.stem(val)
            stemOutputLst.append(stemOutput)
        else:
            stemOutputLst.append(val)
    # updating the vocab dictionary with the new values.        
    collocationUnitsDict[key] =stemOutputLst

***
#  Data feature generation and Vocab  Dictionary Sorting.
list of the dictionary values is been created and then they are sorted in alphabatical order. 
Unit are total- 200 and have vocab 253 which would be out column in parse representation.
***

In [ ]:
#parse representation
listValueVocabDict = list(chain.from_iterable(collocationUnitsDict.values()))
print(len(set(listValueVocabDict)))
print(set(listValueVocabDict))
vocabFinal=[]
vectorizer = CountVectorizer(analyzer = "word")
# data feature of the vocab
dataFeatures = vectorizer.fit_transform([' '.join(value) for value in collocationUnitsDict.values()])
#Final vocab of the units, vocab is sorted in the alphabatical order here.
vocabFinal= vectorizer.get_feature_names()
print("Data shape(Units,Vocab):",dataFeatures.shape)


***
# Indexing of the Vocabulary
This is the final step for vocab generation. Index is been added to each word and thus final vocab dictionary is been generated.
***

In [ ]:
# iterating through final vocab and addinf index.
dictWordIndex={}
#iterate over the vocab list.
for i in range(len(vocabFinal)):
    # adding the idex to the vocab dictionary
    dictWordIndex[vocabFinal[i]]= i

# creating the file object and opening it in append mode
with open('29781965_vocab.txt',"a+") as infile:
    for k,val in dictWordIndex.items():
        strVcab= k+":"+ str(val)+"\n"
        infile.write(strVcab)
print("Dictionary with the Indexes:",dictWordIndex)

***
## Creating the parse Representations for units
Parse represenation for the units is been created, for the vocab which is created by count.
***

In [ ]:
# generate the parse representation
finalOutputList=[]
r=0
c=0
vcabForParse=[]
vcabForParse= list(dictWordIndex.values())

units=list(collocationUnitsDict.keys())
for r in range(len(units)):
    strFinalOutput=units[r] 
    for c in range(len(vocabFinal)):
        strFinalOutput=strFinalOutput +","+str(vcabForParse[c])+":"+ str(dataFeatures.toarray()[r][c])+" "
    finalOutputList.append(strFinalOutput)

# creating the file object and opening it in append mode
with open('29781965_countVec.txt',"a+") as infile:
    for val in finalOutputList:
        print(val)
        infile.write(val)

In [ ]:
# Before wrangling statistics
words = unitTokens
len(words)
vocab = set(words)
lexicalDiversity = len(words)/len(vocab)
print ("Vocabulary size: ",len(vocab),"\nTotal number of tokens: ", len(words), \
"\nLexical diversity: ", lexicalDiversity)
print("hello")
listFinalSummary = list(chain.from_iterable(collocationUnitsDict.values()))
wordAF=len(listFinalSummary)
# After wrangling statistics
len(listFinalSummary)
vocabAF = set(listFinalSummary)
lexicalDiversityAF = len(listFinalSummary)/len(vocabAF)
print ("Vocabulary size: ",len(vocabAF),"\nTotal number of tokens: " ,wordAF,
"\nLexical diversity: ", lexicalDiversityAF)

## Summary

Summary of the data wrangling is been represented below.

######################### Text Statistics Before Wrangling ##################################

Totale no of Words:  31649<br>
Vocabulary size:  4284<br> 
Lexical diversity:  7.387721755368815<br>

######################### Final Text Statistics After Wranling ##################################

Total number of words:12116 <br>
Total number of vocabs: 310<br>
Lexical diversity is : 39.83 %<br>